In [1]:
#fast chekc to see if IP matches 
import requests
print(requests.get("https://api.ipify.org").text)

37.67.198.11


If the video has an available transcript, retrieves the full transcript text using youtube_transcript_api,
otherwise, returns the raised exception if no transcript available

In [7]:
from youtube_transcript_api import YouTubeTranscriptApi
from urllib.parse import urlparse, parse_qs
import logging

In [19]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s [%(name)s] %(message)s",
    filename="yt_api.log",
    filemode="a",  # append
)

logger = logging.getLogger(__name__)

In [20]:
# video_url = "https://www.youtube.com/watch?v=GuM6dQGRFyQ"
# video_url = "https://www.youtube.com/watch?v=-hkEzwaY754"
video_url = "https://www.youtube.com/watch?v=HhzpDM36Pzg"

In [21]:
def get_youtube_video_id(url: str) -> str | None:
    parsed = urlparse(url)

    # youtu.be/<id>
    if parsed.netloc in ("youtu.be", "www.youtu.be"):
        return parsed.path.lstrip("/")

    # youtube.com/*
    if "youtube.com" in parsed.netloc:
        # watch?v=<id>
        if parsed.path == "/watch":
            return parse_qs(parsed.query).get("v", [None])[0]

        # /embed/<id>, /shorts/<id>
        parts = parsed.path.split("/")
        if len(parts) >= 3 and parts[1] in {"embed", "shorts"}:
            return parts[2]

    return None

def retrieve_video_transcript(video_url:str) -> str:
    video_id = get_youtube_video_id(video_url)
    ytt_api = YouTubeTranscriptApi()
    full_text=""
    try : 
        fetched_transcript = ytt_api.fetch(video_id)
    except Exception as e:
        logger.exception(
            "Failed to fetch transcript for video_url=%s",
            video_url,
        )
        return("")
    for snippet in fetched_transcript.snippets:
        full_text+= snippet.text + " "
    return(full_text)

In [22]:
test_text = retrieve_video_transcript(video_url)

In [23]:
test_text

"We're excited to bring Postgress, [music] the world's most popular open-source database to the Snowflake platform. This fully managed solution eliminates the [music] architectural divide between transactional apps and analytical insights [music] by consolidating your data footprint onto a single powerful platform. You can streamline development and drive innovation. >> [music] >> To get started with Snowflake Postgress, begin in your Snow Site window and create a new [music] instance. Give it a name. Choose the instance type. Put the amount of memory and CPUs you'd like and how much storage [music] you need. These can be changed later without having to migrate data. Select if you need a high availability failover and create the instance. You can use the connection string [music] on screen to connect Postgress from a command line or a graphical interface. All standard Postgress [music] features like extensions and permission are available. Here we connect Snowflake Postgress Psql and c